# Initialisation

In [5]:
#Importing all the necessary libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion, Pipeline
from datetime import datetime as dt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as TTS, KFold as K, cross_val_score as score  
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR 
from xgboost import XGBRegressor as XGR
import warnings
warnings.filterwarnings('ignore')
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator

import pickle



In [6]:
#Importing the datset
df = pd.read_csv(r'../Data/flight_price.csv')
target=df.Price

In [7]:
class filters(BaseEstimator,TransformerMixin):
    def __init__(self, Total_Stops):
        self.Total_Stops=Total_Stops

    def fit(self,X,y=None):
        return self

    def transform(self, X, y=None):

        non_stop={'non-stop':1, np.nan:1, '2 stops':0, '1 stop':0, '3 stops':0,'4 stops':0}
        X.Total_Stops = X.Total_Stops.map(non_stop)
        
        X = X[X.Total_Stops==1]

        return X.values


In [8]:
pipe=Pipeline([('filter_hopping_flights', filters('Total_Stops'))])

In [9]:
op=pipe.fit_transform(df)

In [12]:
pd.DataFrame(op).shape

(3492, 11)

In [14]:
df.shape

(10683, 11)

# Preprocessing trial

Tasks:

1. Airline -> OneHotEncoder() `DONE`
2. Date_of_Journey -> Month of year, Day of month, day of week, day of year  `DONE`
3. Source and Destination -> Route  `DONE`  
4. Drop Route, Arrival_Time, Total_Stops
5. Dep_Time -> Extract hour and min `DONE`
6. Dep hour -> Time of day
7. Duration -> Duration hour, duration minutes `DONE`

In [ ]:
df.drop(['Additional_Info','Arrival_Time','Route'],axis=1,inplace=True)

In [ ]:
p2.fit(df)

In [ ]:
pp.transform(df)

In [ ]:
df.Dep_Time

In [ ]:
type(df.Date_of_Journey[0])


In [ ]:
x.

In [ ]:
sns.boxplot(y)

# Model

In [ ]:
def models(model):

    folds=K(n_splits=5, shuffle=True, random_state=4)
    r=score(model,X,y,scoring='neg_mean_squared_error',cv=folds)
    scores = str(-round(np.sqrt(r.mean()),2))+" ± "+str(round(np.sqrt(r.std()),2))
    return scores

In [ ]:
d=DTR(max_depth=14)
r=RFR(max_depth=16, random_state=123456)
x=XGR(max_depth=5,eta=0.5, subsample=0.92)
print("Absolute error for Decision tree regressor",models(d))    #Median best max_depth after 100 iterations of cross validation
print("Absolute error for Random forest regressor",models(r))
print("Absolute error for XGB Regressor",models(x))

In [ ]:
r.fit(X,y)

In [ ]:
rr=DTR(max_depth=16)
rr.fit(X,y)
pd.DataFrame(rr.feature_importances_*100,index=X.columns).sort_values(0,ascending=False).style.background_gradient(cmap='Reds')

# Deployment

In [ ]:

hours_calc=df[['Airline','Source','Destination','Duration_hours','Duration_mins']]
hours_calc=hours_calc[hours_calc.Airline.isin(['IndiGo','Air India','SpiceJet','Air Asia','GoAir','Vistara'])]
hours_calc=hours_calc.groupby(['Airline','Source','Destination'])[['Duration_hours','Duration_mins']].mean().reset_index()

In [ ]:
hours_calc.to_csv(r"../deployment/hour_calculation.csv", index=False)

In [ ]:
with open('..\deployment\model','wb') as f:
    pickle.dump(r,f)